In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

# 초기 설정 및 URL 열기
url = 'https://app.catchtable.co.kr/'
id = '01076300106'
pw = '=Cherrycow910'

driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)

data = []

# 팝업 정리
script = "document.querySelectorAll('#dock > div > nav > a:nth-child(5)').forEach(a => a.click())"
driver.execute_script(script)           
script = "document.querySelectorAll('._1qn1ik44._1ltqxco1e').forEach(a => a.click())"
driver.execute_script(script)
time.sleep(2)
script = "document.querySelectorAll('#main > section > div.__inner > div > div > div._1w07br45 > button:nth-child(3)').forEach(a => a.click())"
driver.execute_script(script)

# 로그인 절차
# ID와 PW 입력
driver.execute_script("document.getElementById('login-id')")
time.sleep(1)
driver.find_element(By.ID, 'login-id').send_keys(id)
time.sleep(2)
driver.execute_script("document.getElementById('login-pw')")
time.sleep(1)
driver.find_element(By.ID, 'login-pw').send_keys(pw)
time.sleep(2)

# 로그인 버튼 클릭
driver.find_element(By.CSS_SELECTOR, '#main > div > button').click()
time.sleep(2)

# 로그인 후 팝업 닫기
driver.find_element(By.CSS_SELECTOR, '._1qn1ik44._1ltqxco1e').click()
time.sleep(4)

# 내 주변 들어가기
driver.find_element(By.CSS_SELECTOR, '#main > section:nth-child(6) > div > div.section-body > div > div > div > a.icon').click()
time.sleep(3)

# 필터 클릭
driver.find_element(By.CSS_SELECTOR, '#main > div.search-header.__autohide > div:nth-child(2) > div > div.speb4g1 > button').click()
time.sleep(3)

# 서울에서 특정 지역 (예: 강남) 필터링
# 적절한 셀렉터는 페이지 구조에 따라 다를 수 있으므로 확인이 필요합니다.
# 예시: 서울 클릭 후 강남 클릭

# 서울클릭
driver.find_element(By.CSS_SELECTOR, '.fldt7j1 > button:nth-child(2)').click()
time.sleep(1)

# 강남클릭
driver.find_element(By.CSS_SELECTOR, '#overlay > div.BottomSheet_bottomSheetContent__1tetrf80.BottomSheet_bottomSheetContent_hasMaxWidth_true__1tetrf81 > div > main > div:nth-child(1) > section > div.fldt7j0 > div.fldt7j2 > button:nth-child(2)').click()
time.sleep(1)

# 결과 보기 클릭
driver.find_element(By.CSS_SELECTOR, '#overlay > div.BottomSheet_bottomSheetContent__1tetrf80.BottomSheet_bottomSheetContent_hasMaxWidth_true__1tetrf81 > div > footer > div.j4ixu30 > div > button').click()
time.sleep(5)

# 총 매장수 확인 및 기타
m_num = driver.page_source
soup = BeautifulSoup(m_num, 'html.parser')
it = soup.select_one('#search-result-header > div._54rkq62 > strong').text
num = int(it.replace(',','').replace('개',''))
idx = 0
region_num = 0

# current_scroll_position = 0
scroll_increment = 450
print(num)
while idx < num:
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select('.restaurant-info')

    for item in items:
        current_scroll_position = 0
        if item is not None:
            # 카테고리 및 지역
            Category = item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags").text.split(' • ')[0] if item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags") else "NA"
            Location = item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags").text.split(' • ')[1] if item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags") else "NA"
            Price_range = item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.price").text if item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.price") else "NA"

            idx +=1

            # 매장 클릭
            driver.find_element(By.CSS_SELECTOR, f'#virtual_{idx} > div > div > div > div').click()
            time.sleep(2)
            
            # 팝업 닫기
            script = "document.querySelectorAll('._1ltqxco1b._10fm75h11').forEach(a => a.click())"
            driver.execute_script(script)
            time.sleep(1)

            # 상세주소
            script = "document.querySelectorAll('._4qv338d.location').forEach(a => a.click())"
            driver.execute_script(script)
            time.sleep(1)
            script = "document.querySelectorAll('.z8ddia').forEach(a => a.click())"
            driver.execute_script(script)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            region = soup.select_one('.addr').text.replace('복사','') if soup.select_one('.addr') else "NA"
            time.sleep(3)
            driver.back()
            time.sleep(3)
            
            script = "document.querySelectorAll('._4qv338e').forEach(a => a.click())"
            driver.execute_script(script)
            time.sleep(2)

            # 매장 정보 및 리뷰 가져오기
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            shop_name = soup.select_one('._17saaqd5._1ltqxco19._1e99eu31').text if soup.select_one("._17saaqd5._1ltqxco19._1e99eu31") else 'NA'
            shop_rating = soup.select_one('.score').text if soup.select_one('.score') else "NA"
            review_num = soup.select_one('#main > div.tab-menu.sticky > ul > li.active > a > span:nth-child(2)').text.replace(',','') if soup.select_one('#main > div.tab-menu.sticky > ul > li.active > a > span:nth-child(2)') else 'NA'
            
            # 리뷰수
            r_num = int(review_num)

            # 리뷰 백개 뽑기
            if r_num >= 100:
                for i in range(100):
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    
                    # 찜
                    save = soup.select_one("#dock > div > span").text if soup.select_one("#dock > div > span") else "NA"
                    # 유저 이름
                    nickname = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span") else "NA"
                    # 유저 별점
                    user_rating = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div") else "NA"
                    # 점심 저녁
                    lun_di = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p") else "NA"
                    # 리뷰 등록시간
                    times = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span") else "NA"
                    # 리뷰
                    review = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p") else "NA"

                    data.append([shop_name,shop_rating,r_num,Category,Location,region,Price_range,nickname,user_rating,lun_di,times,review,save])
                    
                    time.sleep(1)
                    last_height = driver.execute_script("return document.body.scrollHeight")
                    current_scroll_position += scroll_increment
                    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
                    time.sleep(2)


            else :
                for i in range(r_num):
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    
                    # 찜
                    save = soup.select_one("#dock > div > span").text if soup.select_one("#dock > div > span") else "NA"
                    # 유저 이름
                    nickname = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span") else "NA"
                    # 유저 별점
                    user_rating = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div") else "NA"
                    # 점심 저녁
                    lun_di = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p") else "NA"
                    # 리뷰 등록시간
                    times = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span") else "NA"
                    # 리뷰
                    review = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p") else "NA"

                    data.append([shop_name,shop_rating,r_num,Category,Location,region,Price_range,nickname,user_rating,lun_di,times,review,save])
                    time.sleep(1)
                    last_height = driver.execute_script("return document.body.scrollHeight")
                    current_scroll_position += scroll_increment
                    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
                    time.sleep(2) 
            
            driver.back()
            time.sleep(2)
            driver.back()
            time.sleep(2)
            # print(data)
            print(review)
            
driver.quit()

1265
너무 맛있어요!!!!!! 입에서 녹는 촉촉한 식감에 곁들여 먹는 토핑도 다양하게 즐길수 있어서 세조각 순식간에 다 먹었어요.  


ValueError: invalid literal for int() with base 10: 'NA'

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# 초기 설정 및 URL 열기
url = 'https://app.catchtable.co.kr/'
id = '01076300106'
pw = '=Cherrycow910'

driver = webdriver.Chrome()
driver.get(url)

wait = WebDriverWait(driver, 10)

# 팝업 정리
driver.execute_script("document.querySelectorAll('#dock > div > nav > a:nth-child(5)').forEach(a => a.click())")           
driver.execute_script("document.querySelectorAll('._1qn1ik44._1ltqxco1e').forEach(a => a.click())")
time.sleep(2)
driver.execute_script("document.querySelectorAll('#main > section > div.__inner > div > div > div._1w07br45 > button:nth-child(3)').forEach(a => a.click())")

# 로그인 절차
wait.until(EC.presence_of_element_located((By.ID, 'login-id'))).send_keys(id)
wait.until(EC.presence_of_element_located((By.ID, 'login-pw'))).send_keys(pw)
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > div > button'))).click()

# 로그인 후 팝업 닫기
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '._1qn1ik44._1ltqxco1e'))).click()

# 내 주변 들어가기
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > section:nth-child(6) > div > div.section-body > div > div > div > a.icon'))).click()

# 필터 클릭
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > div.search-header.__autohide > div:nth-child(2) > div > div.speb4g1 > button'))).click()

# 서울에서 특정 지역 (예: 강남) 필터링
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.fldt7j1 > button:nth-child(2)'))).click()
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#overlay > div.BottomSheet_bottomSheetContent__1tetrf80.BottomSheet_bottomSheetContent_hasMaxWidth_true__1tetrf81 > div > main > div:nth-child(1) > section > div.fldt7j0 > div.fldt7j2 > button:nth-child(2)'))).click()
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#overlay > div.BottomSheet_bottomSheetContent__1tetrf80.BottomSheet_bottomSheetContent_hasMaxWidth_true__1tetrf81 > div > footer > div.j4ixu30 > div > button'))).click()

time.sleep(5)
# 총 매장수 확인 및 기타
m_num = driver.page_source
soup = BeautifulSoup(m_num, 'html.parser')
num = int(soup.select_one('#search-result-header > div._54rkq62 > strong').text.replace(',','').replace('개',''))
print(num)
idx = 0
# current_scroll_position = 0
scroll_increment = 450
data = []

while idx < num:
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select('.restaurant-info')

    for item in items:
        current_scroll_position = 0
        if item is not None:
            Category = item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags").text.split(' • ')[0] if item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags") else "NA"
            Location = item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags").text.split(' • ')[1] if item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags") else "NA"
            Price_range = item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.price").text if item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.price") else "NA"
            idx += 1

            driver.find_element(By.CSS_SELECTOR, f'#virtual_{idx} > div > div > div > div').click()
            time.sleep(2)

            driver.execute_script("document.querySelectorAll('._1ltqxco1b._10fm75h11').forEach(a => a.click())")
            time.sleep(1)
            driver.execute_script("document.querySelectorAll('._4qv338d.location').forEach(a => a.click())")
            time.sleep(1)
            driver.execute_script("document.querySelectorAll('.z8ddia').forEach(a => a.click())")
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            region = soup.select_one('.addr').text.replace('복사', '') if soup.select_one('.addr') else "NA"
            time.sleep(3)
            driver.back()
            time.sleep(3)
            driver.execute_script("document.querySelectorAll('._4qv338e').forEach(a => a.click())")
            time.sleep(2)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            shop_name = soup.select_one('._17saaqd5._1ltqxco19._1e99eu31').text if soup.select_one("._17saaqd5._1ltqxco19._1e99eu31") else 'NA'
            shop_rating = soup.select_one('.score').text if soup.select_one('.score') else "NA"
            review_num = soup.select_one('#main > div.tab-menu.sticky > ul > li.active > a > span:nth-child(2)').text.replace(',', '') if soup.select_one('#main > div.tab-menu.sticky > ul > li.active > a > span:nth-child(2)') else 'NA'

            r_num = int(review_num)

            for i in range(100):
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                save = soup.select_one("#dock > div > span").text if soup.select_one("#dock > div > span") else "NA"
                nickname = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span") else "NA"
                user_rating = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div") else "NA"
                lun_di = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p") else "NA"
                times = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span") else "NA"
                review = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p") else "NA"

                data.append([shop_name, shop_rating, r_num, Category, Location, region, Price_range, nickname, user_rating, lun_di, times, review, save])
                time.sleep(1)
                last_height = driver.execute_script("return document.body.scrollHeight")
                current_scroll_position += scroll_increment
                driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
                time.sleep(2)

            driver.back()
            time.sleep(2)
            driver.back()
            time.sleep(2)
            print(review)

driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF7349EEEA2+31554]
	(No symbol) [0x00007FF734967ED9]
	(No symbol) [0x00007FF73482872A]
	(No symbol) [0x00007FF7347FD995]
	(No symbol) [0x00007FF7348A44D7]
	(No symbol) [0x00007FF7348BC051]
	(No symbol) [0x00007FF73489CDD3]
	(No symbol) [0x00007FF73486A33B]
	(No symbol) [0x00007FF73486AED1]
	GetHandleVerifier [0x00007FF734CF8B1D+3217341]
	GetHandleVerifier [0x00007FF734D45AE3+3532675]
	GetHandleVerifier [0x00007FF734D3B0E0+3489152]
	GetHandleVerifier [0x00007FF734A9E776+750614]
	(No symbol) [0x00007FF73497375F]
	(No symbol) [0x00007FF73496EB14]
	(No symbol) [0x00007FF73496ECA2]
	(No symbol) [0x00007FF73495E16F]
	BaseThreadInitThunk [0x00007FFF86757344+20]
	RtlUserThreadStart [0x00007FFF868BCC91+33]


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# 초기 설정 및 URL 열기
url = 'https://app.catchtable.co.kr/'
id = '01076300106'
pw = '=Cherrycow910'

# options = webdriver.ChromeOptions()
# options.add_argument('--start-maximized')  # 브라우저 창을 최대화하여 시작

# driver = webdriver.Chrome(options=options)
# driver.get(url)

# # 33% 축소를 위해 창 크기 조정 (화면 해상도의 33% 크기로 설정)
# screen_width = driver.execute_script("return window.screen.width;")
# screen_height = driver.execute_script("return window.screen.height;")

# new_width = int(screen_width * 0.33)
# new_height = int(screen_height * 0.33)

# driver.set_window_size(new_width, new_height)

# 브라우저 열기
driver = webdriver.Chrome()
driver.get(url)

# 페이지 로드 대기
time.sleep(5)

# # 페이지 줌 레벨 33%로 설정
# driver.execute_script("document.body.style.zoom='33%'")

# driver = webdriver.Chrome()
# driver.get(url)

wait = WebDriverWait(driver, 10)

# 팝업 정리
driver.execute_script("document.querySelectorAll('#dock > div > nav > a:nth-child(5)').forEach(a => a.click())")           
driver.execute_script("document.querySelectorAll('._1qn1ik44._1ltqxco1e').forEach(a => a.click())")
time.sleep(2)
driver.execute_script("document.querySelectorAll('#main > section > div.__inner > div > div > div._1w07br45 > button:nth-child(3)').forEach(a => a.click())")

# 로그인 절차
wait.until(EC.presence_of_element_located((By.ID, 'login-id'))).send_keys(id)
wait.until(EC.presence_of_element_located((By.ID, 'login-pw'))).send_keys(pw)
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > div > button'))).click()

# 로그인 후 팝업 닫기
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '._1qn1ik44._1ltqxco1e'))).click()

# 내 주변 들어가기
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > section:nth-child(6) > div > div.section-body > div > div > div > a.icon'))).click()

# 필터 클릭
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > div.search-header.__autohide > div:nth-child(2) > div > div.speb4g1 > button'))).click()

# 서울에서 특정 지역 (예: 강남) 필터링
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.fldt7j1 > button:nth-child(2)'))).click()
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#overlay > div.BottomSheet_bottomSheetContent__1tetrf80.BottomSheet_bottomSheetContent_hasMaxWidth_true__1tetrf81 > div > main > div:nth-child(1) > section > div.fldt7j0 > div.fldt7j2 > button:nth-child(2)'))).click()
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#overlay > div.BottomSheet_bottomSheetContent__1tetrf80.BottomSheet_bottomSheetContent_hasMaxWidth_true__1tetrf81 > div > footer > div.j4ixu30 > div > button'))).click()

time.sleep(5)
# 총 매장수 확인 및 기타
m_num = driver.page_source
soup = BeautifulSoup(m_num, 'html.parser')
num = int(soup.select_one('#search-result-header > div._54rkq62 > strong').text.replace(',','').replace('개',''))
print(num)
idx = 0
# current_scroll_position = 0
scroll_increment = 450
data = []

while idx < num:
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select('.restaurant-info')

    for item in items:
        current_scroll_position = 0
        if item is not None:
            Category = item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags").text.split(' • ')[0] if item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags") else "NA"
            Location = item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags").text.split(' • ')[1] if item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.tags") else "NA"
            Price_range = item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.price").text if item.select_one(f"#virtual_{idx} > div > div > div > div > div > div.detail > div.restaurant-meta > div:nth-child(2) > div > span.price") else "NA"
            idx += 1

            driver.find_element(By.CSS_SELECTOR, f'#virtual_{idx} > div > div > div > div').click()
            time.sleep(2)

            driver.execute_script("document.querySelectorAll('._1ltqxco1b._10fm75h11').forEach(a => a.click())")
            time.sleep(1)
            driver.execute_script("document.querySelectorAll('._4qv338d.location').forEach(a => a.click())")
            time.sleep(1)
            driver.execute_script("document.querySelectorAll('.z8ddia').forEach(a => a.click())")
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            region = soup.select_one('.addr').text.replace('복사', '') if soup.select_one('.addr') else "NA"
            time.sleep(3)
            driver.back()
            time.sleep(3)


            driver.execute_script("document.querySelectorAll('._4qv338e').forEach(a => a.click())")
            time.sleep(2)

            # # 리뷰 100개 먼저 로드
            # for i in range(9):
            #     try:
            #         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            #         html = driver.page_source
            #         soup = BeautifulSoup(html, 'html.parser')
            #     except Exception as e:
            #         print("요소를 다시 찾습니다.")
            #     time.sleep(1)

            # 페이지 줌 레벨 33%로 설정
            driver.execute_script("document.body.style.zoom='33%'")
            time.sleep(5)

        # 리뷰수
            r_num = int(review_num)

            # 리뷰 백개 뽑기
            if r_num >= 100:
                for i in range(100):
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    
                    # 찜
                    save = soup.select_one("#dock > div > span").text if soup.select_one("#dock > div > span") else "NA"
                    # 유저 이름
                    nickname = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span") else "NA"
                    # 유저 별점
                    user_rating = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div") else "NA"
                    # 점심 저녁
                    lun_di = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p") else "NA"
                    # 리뷰 등록시간
                    times = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span") else "NA"
                    # 리뷰
                    review = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p") else "NA"

                    data.append([shop_name,shop_rating,r_num,Category,Location,region,Price_range,nickname,user_rating,lun_di,times,review,save])
                    
                    time.sleep(1)
                    last_height = driver.execute_script("return document.body.scrollHeight")
                    current_scroll_position += scroll_increment
                    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
                    time.sleep(2)
                    print(data)


            else :
                for i in range(r_num):
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    
                    # 찜
                    save = soup.select_one("#dock > div > span").text if soup.select_one("#dock > div > span") else "NA"
                    # 유저 이름
                    nickname = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span") else "NA"
                    # 유저 별점
                    user_rating = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div") else "NA"
                    # 점심 저녁
                    lun_di = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p") else "NA"
                    # 리뷰 등록시간
                    times = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span") else "NA"
                    # 리뷰
                    review = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p") else "NA"

                    data.append([shop_name,shop_rating,r_num,Category,Location,region,Price_range,nickname,user_rating,lun_di,times,review,save])
                    time.sleep(1)
                    last_height = driver.execute_script("return document.body.scrollHeight")
                    current_scroll_position += scroll_increment
                    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
                    time.sleep(2) 
                    print(data)
            
            driver.back()
            time.sleep(2)
            driver.back()
            time.sleep(2)
            # print(data)
            # print(review)


            # html = driver.page_source
            # soup = BeautifulSoup(html, 'html.parser')
            # shop_name = soup.select_one('._17saaqd5._1ltqxco19._1e99eu31').text if soup.select_one("._17saaqd5._1ltqxco19._1e99eu31") else 'NA'
            # shop_rating = soup.select_one('.score').text if soup.select_one('.score') else "NA"
            # review_num = soup.select_one('#main > div.tab-menu.sticky > ul > li.active > a > span:nth-child(2)').text.replace(',', '') if soup.select_one('#main > div.tab-menu.sticky > ul > li.active > a > span:nth-child(2)') else 'NA'

            # r_num = int(review_num)

            # for i in range(100):
            #     html = driver.page_source
            #     soup = BeautifulSoup(html, 'html.parser')
            #     save = soup.select_one("#dock > div > span").text if soup.select_one("#dock > div > span") else "NA"
            #     nickname = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span") else "NA"
            #     user_rating = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div") else "NA"
            #     lun_di = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p") else "NA"
            #     times = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span") else "NA"
            #     review = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p") else "NA"

            #     data.append([shop_name, shop_rating, r_num, Category, Location, region, Price_range, nickname, user_rating, lun_di, times, review, save])
            #     # time.sleep(1)
            #     # last_height = driver.execute_script("return document.body.scrollHeight")
            #     # current_scroll_position += scroll_increment
            #     # driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
            #     time.sleep(2)

            # driver.back()
            # time.sleep(2)
            # driver.back()
            # time.sleep(2)
            # print(review)

driver.quit()


1265
[['제철소', '4.7', 405, '다이닝바', '논현', '서울특별시 강남구 도산대로11길 31-8', '점심 영업안함 · 저녁 1 - 6만원', '마니루다', '4.7 ', '저녁', '2024.06.13', "강남구 신사동 가로수길에 위치한 이자카야 유희 일본어로 '석양'이라는 말처럼 은은한 분위기가 돋보이는 분위기 맛집.  주류가 먼저 나오고 식사와 안주는 순서대로 나와서 음식을 음미하며 여유롭게 먹을 수 있어요 오픈 키친이라 바 테이블에 앉으면 조리하는 과정을 직접 볼 수 있는데, 안주맛집이라 불리는 이유를 알 수 있듯 음식이 다 맛있다.  짐빔 위스키 하이볼과 기린 맥주를 주문하고 유니 트러플 타르트와 유희식 제철 사비체, 가라아케와 참께소스를 주무했는데 하이볼은 청량하고 음식들이 다 진짜 맛있어서 맛있다를 연발하면 먹었어요.  일본어로 석양이라는 말처럼 분위기도 좋아서 밋있는 음식을 즐기고 싶거나 특별한 분위기를 원할 때, 이자카야 유희를 추천해요  ", '6,363']]
[['제철소', '4.7', 405, '다이닝바', '논현', '서울특별시 강남구 도산대로11길 31-8', '점심 영업안함 · 저녁 1 - 6만원', '마니루다', '4.7 ', '저녁', '2024.06.13', "강남구 신사동 가로수길에 위치한 이자카야 유희 일본어로 '석양'이라는 말처럼 은은한 분위기가 돋보이는 분위기 맛집.  주류가 먼저 나오고 식사와 안주는 순서대로 나와서 음식을 음미하며 여유롭게 먹을 수 있어요 오픈 키친이라 바 테이블에 앉으면 조리하는 과정을 직접 볼 수 있는데, 안주맛집이라 불리는 이유를 알 수 있듯 음식이 다 맛있다.  짐빔 위스키 하이볼과 기린 맥주를 주문하고 유니 트러플 타르트와 유희식 제철 사비체, 가라아케와 참께소스를 주무했는데 하이볼은 청량하고 음식들이 다 진짜 맛있어서 맛있다를 연발하면 먹었어요.  일본어로 석양이라는 말처럼 분위기도 좋아서 밋있는 음식을 즐기고 싶거나 특별한 분위기를 원할 때, 이자카야 유희를 추천해요  

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF7349EEEA2+31554]
	(No symbol) [0x00007FF734967ED9]
	(No symbol) [0x00007FF734828559]
	(No symbol) [0x00007FF7348797C2]
	(No symbol) [0x00007FF73486C151]
	(No symbol) [0x00007FF73489D02A]
	(No symbol) [0x00007FF73486BA76]
	(No symbol) [0x00007FF73489D240]
	(No symbol) [0x00007FF7348BC977]
	(No symbol) [0x00007FF73489CDD3]
	(No symbol) [0x00007FF73486A33B]
	(No symbol) [0x00007FF73486AED1]
	GetHandleVerifier [0x00007FF734CF8B1D+3217341]
	GetHandleVerifier [0x00007FF734D45AE3+3532675]
	GetHandleVerifier [0x00007FF734D3B0E0+3489152]
	GetHandleVerifier [0x00007FF734A9E776+750614]
	(No symbol) [0x00007FF73497375F]
	(No symbol) [0x00007FF73496EB14]
	(No symbol) [0x00007FF73496ECA2]
	(No symbol) [0x00007FF73495E16F]
	BaseThreadInitThunk [0x00007FFF86757344+20]
	RtlUserThreadStart [0x00007FFF868BCC91+33]
